In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
import json
from pprint import pprint
import datadict
import functions
import devices
import pandas as pd
import proplot as pplt
import numpy as np
import intakedata as intake

In [18]:
def write_dict_to_json_file(data, file_name):
    """
    Converts a Python dictionary to a JSON-formatted string and writes it to a file.

    :param data: Python dictionary to be converted.
    :param file_name: Name of the file to which the JSON data will be written.
    """
    # Convert the dictionary to a JSON-formatted string
    json_data = json.dumps(data, indent=4)

    # Write the JSON data to a file
    with open(file_name, 'w') as file:
        file.write(json_data)

# Extract Data

In [8]:
data = datadict.DataFile("/Users/jmccarty/GitHub/parametric-bipv-app/static/data/raw_lca_data.json")

# Build Descriptive Files

In [49]:
device_dict = {}

for device in intake.Naming().raw_devices:
    device_dict[intake.Naming().convert_devices[device]] = intake.intake_switcher(device, data.data_dict, intake.Naming().raw_layers, intake.Naming().raw_indicators)

In [ ]:
base_dict = {}

for device in intake.Naming().raw_devices:
    clean_device = intake.Naming().convert_devices[device]
    base_dict[clean_device] = []
    new_data = device_dict[clean_device]
    
    for l in new_data.keys():
        new_entry = {"title":l,
                     "options":{}}
        
        layer_options = list(new_data[l].keys())
        
        layer_option_locations = {}
        for option in layer_options:
            
            locations = list(new_data[l][option].keys())
            layer_option_locations[option] = locations
        new_entry['options'] = layer_option_locations
        base_dict[clean_device].append(new_entry)

In [52]:
write_dict_to_json_file(device_dict, "/Users/jmccarty/GitHub/parametric-bipv-app/static/data/device_impact_data.json")
write_dict_to_json_file(base_dict, "/Users/jmccarty/GitHub/parametric-bipv-app/static/data/layer_data.json")

# Build Data File

In [62]:



t_amb = 25
all_data = {}
self_cons_iterate = np.arange(0.05,1.01+0.01,0.05).round(2)

for device_name in ["monocrystalline", "polycrystalline", "cdte", "cigs", "organic", "perovskite"]:
    print(device_name)
    all_data[device_name] = {}
    for loss_factor in np.arange(0.0,1,0.05).round(2):
        loss_factor_str = f'{loss_factor:.2f}'
        
        all_data[device_name][loss_factor_str] = {}
        my_sys = devices.System(device_name, loss_factor=loss_factor)
        my_sys.n_devices = 1
        impact_value = my_sys.device.default_impact * my_sys.device.module_area

        my_data = {}
        # [0.01] + list(np.arange(0.05,1.01,0.05).round(2))
        for self_cons_factor in self_cons_iterate:
            self_cons_factor_str = f'{self_cons_factor:.2f}'
            all_data[device_name][loss_factor_str][self_cons_factor_str] = {}
            sys_impact = []
            g_eff_range = np.linspace(0,2000,500)#np.arange(1,1000,2)
            for g_eff in g_eff_range:
                # if g_eff==0.:
                
                # get one year
                my_sys.system_lifetime_output(g_eff,t_amb)
                
                # calc year impact
                cumulative_impact = my_sys.system_cumulative_impact(impact_value, self_cons_factor)
                sys_impact.append(cumulative_impact)
            sys_impact[0] = sys_impact[1]
            g_eff_range[0] = g_eff_range[1]
            df = pd.DataFrame({"Impact":sys_impact,
                               "Irradiance":g_eff_range})
            # df = df[df['Impact']<=2]
            # df = df[df['Irradiance']<=1000]#.to_csv(f"/Users/jmccarty/GitHub/parametric-bipv-app/sample_figure/sample_curve_{self_sufficiency_factor}.csv",index=None)
            df = df.round(3)
            all_data[device_name][loss_factor_str][self_cons_factor_str]['Impact'] = df['Impact'].tolist()
            all_data[device_name][loss_factor_str][self_cons_factor_str]['Irradiance'] = df['Irradiance'].tolist()




monocrystalline
polycrystalline
cdte
cigs
organic
perovskite


In [63]:
with open("/Users/jmccarty/GitHub/parametric-bipv-app/static/data/static_curve_data3.json","w") as fp:
    json.dump(all_data, fp)